<a href="https://colab.research.google.com/github/JShyam33/Music-generation/blob/master/music_VAE_2_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#refrences 
"""
1) https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py

2) https://blog.keras.io/building-autoencoders-in-keras.html

3) Music VAE pdf

4) https://colab.research.google.com/drive/1MLxvH1FKGlOFGlZts3duqD2zGXnhBj03#scrollTo=3ynxIJwv7cq5

5) https://github.com/subpath/Keras_music_gereration/blob/master/Music%20gerenation%20with%20Keras%20and%20TF.ipynb
"""
# Encoders
"""
Take input sequence flow through 
bi directional LSTM = 2048 dims

the out put of both direction are concatenated
then pass through 
Latent Dense representation = 512

os output is vector containing mean and sigma of z(latent representation) 
"""
#Decoder
"""
vector z is passed to fully connected followed by tanh activation

now we create conductor that produces U embeddings 
two unidirection LSTM = 1024 , 512

again embeddings are passed through fully connected layers

for bottelneck decoder 
2 layers LSTM = 1024
"""

#parameters
"""
optimizer = Adam
lr = 10e-3 to 10e-5
batch size = 512
loss = crossentropy
time step = 32
"""

# train_data = 726283
# valid_data = 207510
# test_data = 103755

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM , Dropout, Activation, Flatten,Dense,Input,Bidirectional,Lambda,Concatenate
from keras.preprocessing import sequence
from keras.models import Sequential,Model,load_model
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.losses import binary_crossentropy,mse
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
import numpy as np
import os
import sys
import tensorflow as tf
import tqdm

Using TensorFlow backend.


In [0]:
f_path = "../content/drive/My Drive/Music/Models/music_vae.h5"
T = 4 #Timestep

In [0]:
tf.reset_default_graph()
K.clear_session()

In [0]:
!cd '../content/drive/My Drive/Music/train'

In [0]:
data = []
train_data = []
test_data = []

In [0]:
base = '../content/drive/My Drive/Music/'
traindir_path = os.path.join(base,'train/')
testdir_path = os.path.join(base,'test/')
for f in os.listdir(traindir_path):
  mid = MidiFile(traindir_path+f)
  song = []
  for msg in mid:
    masg = msg.dict()
    if not msg.is_meta:
      if 'note' in masg:
        song.append([int(1000*masg['time']),masg['note']])  
  data.append(song)

In [0]:
len(data)

128

In [0]:
train_data = data[:104]
test_data = data[104:]

In [0]:
train_data[0]

In [0]:
len(data[0])

3962

In [0]:
print(len(train_data))
print(len(test_data))

3962
24


In [0]:
#train_data = np.array(train_data)
#test_data = np.array(test_data)



no = []
i = 0
for song in train_data:
  n_song = []
  
  for t in range(0,len(song)-T,T):
    
    n_song.append(song[t:t+T])
  
  train_data[i] = n_song
  i = i + 1
  
  

In [0]:
print((train_data[1]))

[[[0, 64], [0, 73], [0, 46], [0, 55]], [[607, 73], [0, 64], [0, 55], [0, 46]], [[6, 64], [0, 73], [0, 55], [0, 46]], [[213, 73], [0, 64], [0, 46], [0, 55]], [[1, 73], [0, 64], [0, 46], [0, 55]], [[639, 64], [0, 73], [0, 55], [0, 46]], [[6, 64], [0, 73], [0, 55], [0, 46]], [[213, 73], [0, 64], [0, 46], [0, 55]], [[1, 64], [0, 73], [0, 46], [0, 55]], [[320, 73], [0, 64], [0, 55], [0, 46]], [[2, 73], [0, 64], [0, 55], [0, 46]], [[106, 64], [0, 73], [0, 46], [0, 55]], [[0, 64], [0, 73], [0, 46], [0, 55]], [[320, 73], [0, 64], [0, 55], [0, 46]], [[2, 64], [0, 73], [0, 46], [0, 55]], [[106, 73], [0, 64], [0, 55], [0, 46]], [[0, 73], [0, 64], [0, 55], [0, 46]], [[320, 64], [0, 73], [0, 46], [0, 55]], [[2, 64], [0, 73], [0, 46], [0, 55]], [[106, 73], [0, 64], [0, 55], [0, 46]], [[0, 73], [0, 64], [0, 55], [0, 46]], [[320, 64], [0, 73], [0, 46], [0, 55]], [[2, 64], [0, 73], [0, 46], [0, 55]], [[106, 73], [0, 64], [0, 55], [0, 46]], [[0, 64], [0, 73], [0, 55], [0, 46]], [[0, 73], [0, 64], [0, 46

In [0]:
train_data = np.array(train_data)
test_data = np.array(test_data)

In [0]:
def sampling(args = [0,0]):
  mean,sigma = args
  batch = K.shape(sigma)[0]
  dim = K.int_shape(sigma)[1]
  epsilon = K.random_normal(shape=(batch,dim,2))
  return mean + K.exp(sigma*0.5)*epsilon

In [0]:

in_music = Input(shape = (T,2))
  
  #bidirectional encoder
print("in_music : ",in_music.get_shape().as_list())
biLSTM_1 = Bidirectional(LSTM(2048,input_shape=(4,2),return_sequences = True))(in_music)
biLSTM_2 = Bidirectional(LSTM(2048,input_shape=(4,2),return_sequences = True))(in_music)
  
  #concatenate the results 
print("2 layer LSTM : ",biLSTM_1.get_shape().as_list())
concatenate = Concatenate()([biLSTM_1 , biLSTM_2])
  
  #converting into latent dimension producing mean and sigma
print("Concatenation : ",concatenate.get_shape().as_list())
latent_dim_mean = Dense(2)(concatenate)
latent_dim_log_sigma = Dense(2)(concatenate)

  #sampling z vector from distribution
print("Latent representation : ",latent_dim_log_sigma.get_shape().as_list())
z = Lambda(sampling, output_shape=(T,2,), name='z')([latent_dim_mean, latent_dim_log_sigma])
print("Z = ",z.get_shape().as_list())
encoder = Model(in_music,[latent_dim_mean,latent_dim_log_sigma,z])
  

#print(z.get_shape().as_list())


  



in_music :  [None, 4, 2]

2 layer LSTM :  [None, None, 4096]
Concatenation :  [None, None, 8192]
Latent representation :  [None, 4, 2]

Z =  [None, 4, 2]


In [0]:
z = Input(shape = (T,2))
print("z input : ",z.get_shape().as_list())
dense = Dense(512)(z)

print("dens representation : ",dense.get_shape().as_list())
  
  #decoder
decode_LSTM1 = LSTM(1024,return_sequences=True)(dense)
decode_LSTM2 = LSTM(1024,return_sequences=True)(decode_LSTM1)
 
print("2 layer LSTM output : ",decode_LSTM1.get_shape().as_list())
output = Dense(2,activation='linear')(decode_LSTM2)
print("output : ",output.get_shape().as_list())
decoder = Model(z,output)

z input :  [None, 4, 2]
dens representation :  [None, 4, 512]
2 layer LSTM output :  [None, None, 1024]
output :  [None, 4, 2]


In [0]:
_#encoder = Music_encoder()
encoder.summary()
#decoder = Music_decoder()
decoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 2)         0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 4, 4096)      33603584    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 4, 4096)      33603584    input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 4, 8192)      0           bidirectional_1[0][0]            
                                                                 bidirectional_2[0][0]      

In [0]:
outputs = decoder(encoder(in_music)[2])
music_vae = Model(in_music,outputs)
music_vae.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 2)              0         
_________________________________________________________________
model_1 (Model)              [(None, 4, 2), (None, 4,  67239940  
_________________________________________________________________
model_2 (Model)              (None, 4, 2)              14691842  
Total params: 81,931,782
Trainable params: 81,931,782
Non-trainable params: 0
_________________________________________________________________


In [0]:
reconstruction_loss = mse(in_music,outputs)
print(reconstruction_loss.shape)
kl = 1 + latent_dim_log_sigma - K.square(latent_dim_mean) - K.exp(latent_dim_log_sigma)
kl = K.sum(kl,axis=-1)
kl *= -0.5
print(kl.shape)
vae_loss =  K.mean(reconstruction_loss + kl)
print(vae_loss.shape)
music_vae.add_loss(vae_loss)
music_vae.compile(optimizer=Adam(lr=0.001))
music_vae.summary()

(?, 4)
(?, 4)
()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 2)              0         
_________________________________________________________________
model_1 (Model)              [(None, 4, 2), (None, 4,  67239940  
_________________________________________________________________
model_2 (Model)              (None, 4, 2)              14691842  
Total params: 81,931,782
Trainable params: 81,931,782
Non-trainable params: 0
_________________________________________________________________


In [0]:
music_vae.load_weights(f_path) 

In [0]:
for i,song in enumerate(train_data):
  print("trainging song ",i)
  music_vae.fit(np.array(song),epochs = 20,batch_size = 100,verbose=1)


trainging song  0
Epoch 1/20
990/990 [==============================] - 2s 2ms/step - loss: 1282.1853
Epoch 2/20
990/990 [==============================] - 2s 2ms/step - loss: 685.8166
Epoch 3/20
990/990 [==============================] - 2s 2ms/step - loss: 427.1163
Epoch 4/20
990/990 [==============================] - 2s 2ms/step - loss: 310.2848
Epoch 5/20
990/990 [==============================] - 2s 2ms/step - loss: 215.0077
Epoch 6/20
990/990 [==============================] - 2s 2ms/step - loss: 149.0221
Epoch 7/20
990/990 [==============================] - 2s 2ms/step - loss: 102.2884
Epoch 8/20
990/990 [==============================] - 2s 2ms/step - loss: 64.4482
Epoch 9/20
990/990 [==============================] - 2s 2ms/step - loss: 49.5356
Epoch 10/20
990/990 [==============================] - 2s 2ms/step - loss: 56.3705
Epoch 11/20
990/990 [==============================] - 2s 2ms/step - loss: 61.9086
Epoch 12/20
990/990 [==============================] - 2s 2ms/step - l

In [0]:
music_vae.save(f_path)

In [0]:
evaluation = music_vae.evaluate(np.array(test_data),batch_size=100,verbose=1)

92389/92389 [==============================] - 122s 1ms/step


In [0]:
print(evaluation)

392.9177751410766


In [0]:
def rescale(values, new_min = 0, new_max = 127):
    output = []
    old_min, old_max = np.max(values), np.min(values)

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

Predicting on given music sequence

In [0]:
music_vae = load_model(f_path)

ValueError: ignored

In [0]:
z = MidiFile('chp_op18.mid')
orig_no = []
time = []

for msg in z:
  masg = msg.dict()
  
  if not msg.is_meta:
    if 'note' in masg:
        orig_no.append([int(1000*masg['time']),masg['note']])
    
z_notes = []
for i in range(0,len(orig_no)-T,T):
  z_notes.append(orig_no[i:i+T])

  

In [0]:
len(z_notes)

2207

In [0]:
z_notes.append(orig_no[-8:])

In [0]:
no = np.array(no)
np.sum(no,axis=0)
tot_time = 154917

In [0]:
z_notes = np.array(z_notes)
z_notes.shape
#z_notes = np.reshape(z_notes,(z_notes.shape[0],z_notes.shape[1],1))

(2207, 4, 2)

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(z_notes).reshape(-1,1))
#prediction = [int(i) for i in prediction]

MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
prediction = music_vae.predict(np.array(z_notes))
#prediction = np.squeeze(prediction)


In [0]:
prediction.shape

(2207, 4, 2)

In [0]:
predicted_song = []

for d in prediction:
  for s in d:
    predicted_song.append(s)


In [0]:
predicted_song

In [0]:
prediction = np.reshape(prediction,(5952,2))
prediction.shape

In [0]:
pred_time = []
pred_note = []

for pred in prediction:
  for p in pred:
    pred_time.append(p[0])
    pred_note.append(p[1])

pred_note = rescale(pred_note)

#prediction = [int(i) for i in prediction]

In [0]:
len(pred_time)

12096

In [0]:
for pred in prediction:
  print(pred)

In [0]:
type_msg = {0:'note_off',1:'note_on'}
pred_type = []
for pred in prediction:
  if pred[0] > 0.5:
    pred_type.append(type_msg[1])
  else:
    pred_type.append(type_msg[0])

In [0]:
len(orig_no)

12104

In [0]:
pred_note

In [0]:
mid = MidiFile()
track = MidiTrack()
p_time = []
#print(prediction)
for n in range(len(prediction)):
  
  t = abs(int(pred_time[n]))
  p_time.append(t)
  no = int(pred_note[n])
  m = np.asarray([147,no,67])
  bytes = m.astype(int)
  msg = Message.from_bytes(bytes[0:3])
  msg.time = t
  track.append(msg)

mid.tracks.append(track)
mid.save('music_vae_v3_chp_op18.mid')

In [0]:
mid.save('../content/drive/My Drive/Music/Models/music_vae_v3_chp_op18.mid')